In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
  inflating: all_applic.dat          
  inflating: all_meta.dat            
  inflating: readme.md               


In [ ]:
!cat all_meta.dat | head -5

*$*
Дата выхода: 2023-04-11
Установлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы
(Контингент: CGN-20180529004)
Первый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян (заявление No CGN-20180529004)


In [ ]:
import re
import torch
import numpy as np
import pandas as pd

from typing import List, Tuple, Dict, Any
from tqdm.notebook import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification

pd.set_option('display.max_colwidth', 200)


def extract_CGN(string: str) -> str:
    # codes: List[str] = re.findall("CGN-.*\s*[0-9]+[A-Z]*[0-9]*", string)
    codes: List[str] = re.findall("[0-9]{9,}[A-Z]*[0-9]*", string)
    if len(codes) == 0:
        return ""
    code: str = re.findall("[0-9]+\S+", codes[0])[0]
    return code


with open("/content/all_applic.dat") as f:
    all_applic = f.read()
all_applic = all_applic.split("*$*")[3:]

docs = list(map(lambda x: x.split("\n")[2], all_applic))
texts = list(map(lambda x: "\n".join(x.split("\n")[3:]), all_applic))

df = pd.DataFrame(data=list(zip(docs, texts)), columns=["docs",  "text"])
df["text"] = df["text"].str.replace("&gt;", "")
df["text"] = df["text"].str.replace("&lt;", "")
df["text"] = df["text"].str.replace("&quot;", "")
mask = df["text"].str.len() > 10
df = df.loc[mask].reset_index(drop=True)


df["docs"] = df["docs"].apply(lambda x: re.findall("[0-9]+-[0-9]+", x)[0])
df["CGN"] = df["text"].apply(extract_CGN)


# df_meta = pd.read_excel("/content/result.xlsx")
# mask = df_meta["Контингент:"].isna()
# df_meta.loc[mask] = "nan"
# df_meta["CGN"] = df_meta["Контингент:"].apply(extract_CGN)
# res = pd.merge(df, df_meta, left_on="CGN", right_on="CGN", how="inner")

In [ ]:
all_applic[:2]

['\n20230731_12345\nКитайский документ 8-8trans.txt\nСиньцзян-Китайская база экологически чистой энергии в Синьцзяне, Лоуп, 1 млн. кВт оптико-волнового поля PC\n(Контингент: CGN-202307050014)\nТерритория, в которой находятся тендерные проекты: Синьцзян-Уйгурский автономный район и Тэгу, округ Лоуп\nУсловия торгов\nВ рамках этого проекта на базе экологически чистой энергетики в Синьцзяне в Лоупе 1 млн. кВт (проект тендера)\nНомер: CGN-202307050014, одобренный подразделением по утверждению проектов, финансируется за счет средств, полученных от предприятия, и реализовано, торгующее\nChina Solar Energy Successing Ltd. В рамках этого проекта уже имеются условия для проведения торгов и в настоящее время проводятся открытые торги.\nII. ОБЩИЙ ОБЗОР ПРОЕКТОВ, ОГРАНИЧЕННОСТИ И ПЛАНИРОВАНИЯ ПРОЕКТОВ\nРазмер проекта:\nСиньцзян-цзян, Китай, Новая энергетическая база мощностью 1 000 000 000 000 000 000 000 кигаваттских люминесцентных оптических дисков в районе Лоуп\nМесто планирования расположено в 

In [ ]:
# D = {
#     "I.": "Уведомление",
#     "II.": "Общий обзор проектов и ограничений",
#     "III.": "Требования к кандидатам",
#     "IV.": [
#         "ПРЕДСТАВЛЕНИЕ РЕКОМЕНДАЦИЙ ПО ПРОГРАММАМ",
#         "ПОЛУЧЕНИЕ ДОКУМЕНТОВ",
#         "ПРЕДСТАВЛЕНИЕ ДОКУМЕНТОВ",
#         "ПРЕДСТАВЛЕНИЕ ДОКУМЕНТОВ ПО ПРОГРАММАМ",
#     ],

#     # "Доступ к документам для посещения программ",
# }

SYMBOLS = ["II.", "III.", "IV.", "  ", "VI.", "VII", "VIII.", "IX"]

def parse_texts(texts: List[str]) -> Dict[int, Dict[str, str]]:
    res: Dict[int, Dict[str, str]] = {}
    for i, text in enumerate(texts):
        res[i] = parse_text(text)
    return res


def parse_text(text: str) -> Dict[str, str]:
    res: Dict[str, str] = {}
    rows = text.split("\n")
    cur = rows[0]
    tmp: List[str] = []
    for row in rows:
        if len(row) == 0:
            continue
        if row.split()[0] in SYMBOLS:
            res[cur] = "\n".join(tmp)
            tmp = []
            cur = row
        else:
            tmp.append(row)
    res[cur] = "\n".join(tmp)
    return res


In [ ]:
def merge_text(tokens: List[Dict[str, Any]]):
    if len(tokens) == 0:
        return {}
    entities: Dict[str, List[str]] = {}
    start = tokens[0]["start"]
    end = tokens[0]["end"]
    entity = tokens[0]["entity"]
    for token in tokens[1:]:
        if end >= token["start"] - 1:
            end = token["end"]
        else:
            if entity in entities:
                entities[entity].append((start, end))
            else:
                entities[entity] = [(start, end)]
            start = token["start"]
            end = token["end"]
            entity = token["entity"]
    if entity in entities:
        entities[entity].append((start, end))
    else:
        entities[entity] = [(start, end)]
    return entities


In [ ]:
def normalize_text(text: str) -> str:
    text = re.sub("\n", ". ", text)
    text = re.sub("\s+", " ", text)
    return text


def get_loc_org(data: List[Dict[str, str]], classifier: TokenClassificationPipeline) -> Tuple[List[str], List[str]]:
    res = []
    for id_doc, doc in tqdm(data.items()):
        glav_name, content = list(doc.items())[0]
        content = normalize_text(content)
        out = classifier(content)
        out = merge_text(out)
        d_tmp = {}
        for type_entity, entities in out.items():
            tmp = []
            for entity in entities:
                text_entity = content[entity[0]:entity[1]]
                tmp.append(text_entity)
            d_tmp[type_entity] = ",".join(set(tmp))
        res.append(d_tmp)
    loc = list(map(lambda x: x["I-LOC"] if "I-LOC" in x else "", res))
    org = list(map(lambda x: x["I-ORG"] if "I-ORG" in x else "", res))
    return loc, org

In [ ]:
XLMRobertaTokenizerFast, XLMRobertaForTokenClassification

(transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast,
 transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForTokenClassification)

In [ ]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier = pipeline("ner", model=model, tokenizer=tokenizer, device=device)

data = parse_texts(df["text"].tolist())

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
loc, org = get_loc_org(data, classifier)

df["location"] = loc
df["organization"] = org


  0%|          | 0/1422 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
df[["location", "organization", "CGN"]].head(30)

,location,organization,CGN
0,"Лоупе,Лоуп,Тэгу,район,Синьцзяне,Синьцзян-Уйгурский",China Solar Energy Successing Ltd,202307050014
1,"Хубэй,Хуанган,Чун","Хубэй,Чайна нью-Энтерпрайз",202301310002
2,"Гуандун,Янцзы,Яньцзян",Jiangian,202305150006
3,Шаньдун,"источник,Чайна Хинси",20230307002
4,Гуандун,Чайна инжиниринг лтд,2021223002
5,Шаньдун,"SK,Чайна Инжиниринг лимитед",202304270004
6,Шаньдун,Чайна Хинэ,20230609001
7,"Гуандун,Цзянмен",Тайшань АЭС,20221122001
8,"Шэньчжэнь,Гуандун",China Institute of Nuclear,2023024005
9,"Гуандун,Янцзы",Jiangian,20230526004
